In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
import os

### Prepare Dataset

In [3]:
# download coco dataset in colab runtime memory
os.makedirs('data/coco')
os.makedirs('data/coco/annotations')

In [4]:
cd data/coco

/content/data/coco


In [5]:
!wget http://images.cocodataset.org/zips/train2017.zip
!wget http://images.cocodataset.org/zips/val2017.zip
#!wget http://images.cocodataset.org/zips/test2017.zip

--2023-12-08 01:27:37--  http://images.cocodataset.org/zips/train2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 54.231.134.25, 52.217.104.244, 52.217.229.9, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|54.231.134.25|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19336861798 (18G) [application/zip]
Saving to: ‘train2017.zip’

train2017.zip       100%[===================>]  18.01G  48.5MB/s    in 7m 1s   

2023-12-08 01:34:38 (43.9 MB/s) - ‘train2017.zip’ saved [19336861798/19336861798]

--2023-12-08 01:34:38--  http://images.cocodataset.org/zips/val2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.216.240.212, 16.182.39.129, 52.217.229.65, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|52.216.240.212|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 815585330 (778M) [application/zip]
Saving to: ‘val2017.zip’

val2017.zip         100%[===================>

In [6]:
!unzip -qq "/content/data/coco/train2017.zip"
!unzip -qq "/content/data/coco/val2017.zip"
#!unzip -qq "/content/coco/test2017.zip"


### CLIP

In [7]:
cd /content/drive/MyDrive/Colab Notebooks/DeepLearning/DL_final_project/DL-final_project/CLIP_prefix_caption

/content/drive/MyDrive/Colab Notebooks/DeepLearning/DL_final_project/DL-final_project/CLIP_prefix_caption


In [8]:
!pip install openai-clip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 9.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 6.1 MB/s eta 0:00:00
  Created wheel for openai-clip: filename=openai_clip-1.0.1-py3-none-any.whl size=1368605 sha256=4075a9adb8fca5b16d865c840d6b80aa939c0107f27db927294d89dd29043cd5
  Stored in directory: /root/.cache/pip/wheels/08/77/8e/8d2f862df6bf7fb4e2007062d2cbaeae49862ec7b56d041229
Successfully built openai-clip


In [9]:
import clip

In [10]:
clip.available_models()

['RN50',
 'RN101',
 'RN50x4',
 'RN50x16',
 'RN50x64',
 'ViT-B/32',
 'ViT-B/16',
 'ViT-L/14',
 'ViT-L/14@336px']

In [ ]:
# Pre Trained 된 CLIP 모델을 이용, 이미지를 CLIP Encoder 통과
# clip model ViT-B/32 is used in ClipCap paper

!python parse_coco.py --clip_model_type ViT-B/32 --data_path '/content/data/coco' --out_path '/content/drive/MyDrive/Colab Notebooks/DeepLearning/DL_final_project/DL-final_project/CLIP_prefix_caption/data/coco' --start_index 560000

### Train Prefix Network

In [ ]:
!python train.py --data ./data/coco/oscar_split_ViT-B_32_train.pkl --out_dir ./coco_train/ --only_prefix --mapping_type transformer --num_layers 8

vocab.json: 100% 1.04M/1.04M [00:00<00:00, 3.87MB/s]
merges.txt: 100% 456k/456k [00:00<00:00, 3.25MB/s]
tokenizer.json: 100% 1.36M/1.36M [00:00<00:00, 4.16MB/s]
config.json: 100% 665/665 [00:00<00:00, 3.44MB/s]
Data size is 566747
model.safetensors: 100% 548M/548M [00:05<00:00, 103MB/s]
generation_config.json: 100% 124/124 [00:00<00:00, 717kB/s]
Train only prefix
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
>>> Training epoch 0
coco_prefix:   2% 314/14337 [02:48<2:07:34,  1.83it/s, loss=4.4]

In [ ]:
!python train.py --only_prefix --data ./data/coco/oscar_split_ViT-B_32_train.pkl --out_dir ./coco_train/ --mapping_type transformer  --num_layres 8 --prefix_length 40 --prefix_length_clip 40